In [1]:
from EMS import EMS
from Organization import Organization
from Energy import Energy
from Hierarchy import Hierarchy
from Dataflow import DataFlow
import pandas as pd
import numpy as np
import pandas as pd

from EMS import EMS
from Organization import Organization
from Energy import Energy
from Hierarchy import Hierarchy
from Dataflow import DataFlow

import xml.etree.ElementTree as ET
from common import readOption, logger
from IPython.display import display
# Set the maximum number of columns to be displayed to None
pd.set_option('display.max_columns', None)

In [2]:
code = readOption("code")
components_binding = readOption("components_binding")        
host=readOption("databases.organization.host")
port=readOption("databases.organization.port")
database=readOption("databases.organization.database")
user=readOption("databases.organization.username")
password=readOption("databases.organization.password")
org = Organization(host=host, port=port, user=user, password=password, database=database)

tenant = org.Tenant(code)

print(f"{tenant.name.values[0]} | {tenant.company_code.values[0]} | {tenant.id.values[0]}")

Canadian International School | cdnis | 9012847f-290f-5e49-a1c7-d9fa78f9eb3e


In [3]:
# generate tenant tree in dataflow
dataFlow = DataFlow(code, components_binding, simulation=True)
df = dataFlow.PreparingData()
# df = dataFlow.LoadData()
logger.info("====================== create_nodes_and_datapoints ==========================")
dataFlow.create_nodes_and_datapoints(df, tenant.id.values[0])

In [ ]:
hr = dataFlow.hr
logger.debug("====================== Simulate TenantTree XML ==========================")    
tenantTree = dataFlow.hr.TenantSimulateTree(tenant.id.values[0],
                        tenant.name.values[0],
                        tenant.company_code.values[0])

site_path = f"./output/{code}"
hr.SaveToXml(tenantTree, f"{site_path}/sim_new_{code}.xml") 

In [3]:
ems = EMS("localhost", "5432", "mangopie", "mangopie", "mangopie")

In [4]:
company_df = ems.company(code="cdnis")
print(company_df)

    id                           name   code
0  142  Canadian International School  cdnis


In [5]:
org = Organization()
tenant_df = org.Tenant(code="cdnis")

print(tenant_df)

                                     id                           name  \
0  9012847f-290f-5e49-a1c7-d9fa78f9eb3e  Canadian International School   

  company_code  
0        cdnis  


In [6]:
joined_tenant = pd.merge(company_df, tenant_df, how="left", left_on="code", right_on="company_code")
tenant_df = joined_tenant[['id_x', 'id_y', 'name_x', 'code']]

new_names = {'id_x': 'company_id', 'id_y': 'tenant_id', 'name_x': 'company', "code": 'company_code'}
tenant_df = tenant_df.rename(columns=new_names)

with pd.option_context('display.max_columns', None):
    display(tenant_df[:3])   

,company_id,tenant_id,company,company_code
0,142,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis


In [7]:
sys_df = ems.systems(code="cdnis")
# Set options temporarily within the context of `display()`
with pd.option_context('display.max_columns', None):
    display(sys_df[:8])

,id,parent_system_id,name,source_key,meter_id,composition_expression,component_of_id,company_id
0,363,NaN,Canadian International School,None,NaN,None,NaN,142
1,379,367.0,Classrooms,,NaN,None,NaN,142
2,364,363.0,Leo Lee Arts Centre,,NaN,None,NaN,142
3,384,NaN,MR,MR,49.0,None,381.0,142
4,385,379.0,Upper Sch CR & Hallways (5-8F),,NaN,1*{id_389} + 1*{id_387} + -1*{id_388} + 1*{id_...,NaN,142
5,375,NaN,1000a,1000a,50.0,None,368.0,142
6,376,NaN,1250a,1250a,50.0,None,368.0,142
7,374,NaN,630a,630a,50.0,None,368.0,142


In [8]:
# sys_df = ems.virtual_systems(code="cdnis")
# # Set options temporarily within the context of `display()`
# with pd.option_context('display.max_columns', None):
#     display(sys_df[:8])

In [9]:
energy = Energy("localhost", "5432", "energy", "energy", "energy")

In [10]:
dp = energy.dataPoint(columns = ["id", "ref_id", "name"])
dp[:3]

,id,ref_id,name
0,e7b7c78b-4ec2-4d75-b7c6-5c25e6ef61c5,520,Special Block AC
1,9a6c367b-5574-47be-bdb1-0ee6838ac61c,16053,socket and other power 23F
2,35a562bf-0e4e-47aa-95cd-76f6cafc0aaf,14463,isf_4NRLP8_4100


In [11]:
joined_df = pd.merge(sys_df, dp, how="left", left_on="id", right_on="ref_id")


# 查看连接后的DataFrame
with pd.option_context('display.max_columns', None):
    display(joined_df[:8])


,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,id_y,ref_id,name_y
0,363,NaN,Canadian International School,None,NaN,None,NaN,142,NaN,NaN,NaN
1,379,367.0,Classrooms,,NaN,None,NaN,142,NaN,NaN,NaN
2,364,363.0,Leo Lee Arts Centre,,NaN,None,NaN,142,NaN,NaN,NaN
3,384,NaN,MR,MR,49.0,None,381.0,142,NaN,NaN,NaN
4,385,379.0,Upper Sch CR & Hallways (5-8F),,NaN,1*{id_389} + 1*{id_387} + -1*{id_388} + 1*{id_...,NaN,142,7436449a-c21b-4a79-8d0d-f1934afe734d,385.0,
5,375,NaN,1000a,1000a,50.0,None,368.0,142,cd445559-331a-4730-8d6b-de1558e90f4c,375.0,1000a
6,376,NaN,1250a,1250a,50.0,None,368.0,142,5ed29e24-b5d2-4b53-b486-ec5a707c03f0,376.0,1250a
7,374,NaN,630a,630a,50.0,None,368.0,142,3ccd5b3c-e3db-49e8-ad08-51b9be9e673a,374.0,630a


In [26]:
joined_df.loc[joined_df['id_x'] == int(386)]#[['id_y', 'ref_id']]

,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,id_y,ref_id,name_y
19,386,NaN,Upper Sch CR & Hallways (5-8F),ML,49.0,None,385.0,142,NaN,NaN,NaN


In [27]:
joined_df.loc[joined_df['source_key'] == 'ML']

,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,id_y,ref_id,name_y
11,371,NaN,ML,ML,49.0,None,368.0,142,c3387410-6c7a-471d-a9fa-45afe64d468e,371.0,ML
19,386,NaN,Upper Sch CR & Hallways (5-8F),ML,49.0,None,385.0,142,NaN,NaN,NaN


In [12]:
with pd.option_context('display.max_columns', None):
    display(tenant_df[:3])   

,company_id,tenant_id,company,company_code
0,142,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis


In [13]:
joined_df = pd.merge(joined_df, tenant_df, how="left", left_on="company_id", right_on="company_id")
# 查看连接后的DataFrame
with pd.option_context('display.max_columns', None):
    display(joined_df[:8])

,id_x,parent_system_id,name_x,source_key,meter_id,composition_expression,component_of_id,company_id,id_y,ref_id,name_y,tenant_id,company,company_code
0,363,NaN,Canadian International School,None,NaN,None,NaN,142,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
1,379,367.0,Classrooms,,NaN,None,NaN,142,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
2,364,363.0,Leo Lee Arts Centre,,NaN,None,NaN,142,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
3,384,NaN,MR,MR,49.0,None,381.0,142,NaN,NaN,NaN,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
4,385,379.0,Upper Sch CR & Hallways (5-8F),,NaN,1*{id_389} + 1*{id_387} + -1*{id_388} + 1*{id_...,NaN,142,7436449a-c21b-4a79-8d0d-f1934afe734d,385.0,,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
5,375,NaN,1000a,1000a,50.0,None,368.0,142,cd445559-331a-4730-8d6b-de1558e90f4c,375.0,1000a,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
6,376,NaN,1250a,1250a,50.0,None,368.0,142,5ed29e24-b5d2-4b53-b486-ec5a707c03f0,376.0,1250a,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis
7,374,NaN,630a,630a,50.0,None,368.0,142,3ccd5b3c-e3db-49e8-ad08-51b9be9e673a,374.0,630a,9012847f-290f-5e49-a1c7-d9fa78f9eb3e,Canadian International School,cdnis


In [14]:
def replace_id(df):
  """
  循环替换 DataFrame 中的 {id_xxx}

  Args:
    df: 输入 DataFrame

  Returns:
    替换后的 DataFrame
  """
  import re
  for i in range(df.shape[0]):
    row = df.iloc[i]
    composition_expression = row['composition_expression']
    for match in re.finditer(r'{id_(\d+)}', composition_expression):
      id_xxx = match.group(1)
      use_system_id = df.loc[df['id'] == int(id_xxx), 'use_system_id'].values[0]
      composition_expression = composition_expression.replace(match.group(0), "{id_"+str(use_system_id)+"}")
    df.loc[i, 'composition_expression'] = composition_expression

  return df

# 使用示例
df = pd.DataFrame({
  'id': [13, 63, 8],
  'composition_expression': [
    '1 if  {id_13}  >= 3 else 1 if  {id_63}  >= 3 else {id_8}',
    '1 if ({id_63}*2)+({id_63}*{id_8})  >= 3 else {id_8}',
    '{id_8}'
  ],
  'use_system_id': [100, 200, 300]
})

print(df)
print("----------------------------------------------------------------")
df = replace_id(df)

print(df)


   id                             composition_expression  use_system_id
0  13  1 if  {id_13}  >= 3 else 1 if  {id_63}  >= 3 e...            100
1  63  1 if ({id_63}*2)+({id_63}*{id_8})  >= 3 else {...            200
2   8                                             {id_8}            300
----------------------------------------------------------------
   id                             composition_expression  use_system_id
0  13  1 if  {id_100}  >= 3 else 1 if  {id_200}  >= 3...            100
1  63  1 if ({id_200}*2)+({id_200}*{id_300})  >= 3 el...            200
2   8                                           {id_300}            300


In [15]:
vdp = energy.virtualDataPoint()
vdp[:3]

,datapoint_id,expression,name
0,4b4a78d4-7bce-415f-b7f3-87faf3248b8d,{id_6002}-{id_5993}-{id_5994}-{id_5996}-{id_59...,HP-3
1,a271606e-e62a-4d42-9fdf-bf4ac890cec4,{id_514}+{id_515},3/F Lights & Sockets
2,4c5d9f40-b2d4-44af-a896-fcb0bf9409b3,na,test


In [16]:
sys_df

,id,parent_system_id,name,source_key,meter_id,composition_expression,component_of_id,company_id
0,363,NaN,Canadian International School,None,NaN,None,NaN,142
1,379,367.0,Classrooms,,NaN,None,NaN,142
2,364,363.0,Leo Lee Arts Centre,,NaN,None,NaN,142
3,384,NaN,MR,MR,49.0,None,381.0,142
4,385,379.0,Upper Sch CR & Hallways (5-8F),,NaN,1*{id_389} + 1*{id_387} + -1*{id_388} + 1*{id_...,NaN,142
5,375,NaN,1000a,1000a,50.0,None,368.0,142
6,376,NaN,1250a,1250a,50.0,None,368.0,142
7,374,NaN,630a,630a,50.0,None,368.0,142
8,370,NaN,B,B,49.0,None,368.0,142
9,377,NaN,Canteen3,Canteen3,76.0,None,368.0,142


In [17]:
hr = Hierarchy()

In [18]:
n_dp = hr.nodeDataPoint()

with pd.option_context('display.max_columns', None):
    display(n_dp[:2])

,id,name,data_type,data_id


In [19]:
n_pov = hr.nodePov()
with pd.option_context('display.max_columns', None):
    display(n_pov[:2])

,id,name


In [20]:
n_site = hr.nodeSite()
with pd.option_context('display.max_columns', None):
    display(n_site[:2])

,id,city_id,name


In [21]:
n_relations = hr.nodeRelations()
with pd.option_context('display.max_columns', None):
    display(n_relations[:2])

,id,parent_id,parent_type,child_id,child_type


In [22]:
# tenantTree = hr.TenantTree("c93789d6-d78e-5382-8817-c3917ca879be")
# tenantTree